In [ ]:
import wandb
import os 

os.environ['WANDB_NOTEBOOK_NAME'] =
%env WANDB_API_KEY = 

wandb.login()

In [2]:
import time
from torch.nn import BCEWithLogitsLoss
import torch
import albumentations as A
from albumentations.pytorch import ToTensorV2
import matplotlib.pyplot as plt
import gc  


INFO:albumentations.check_version:A new version of Albumentations is available: 1.4.18 (you have 1.4.12). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.


In [3]:
from utils import (
    save_checkpoint,
    load_checkpoint,
    check_accuracy,
    save_test_images,
    check_accuracy_val,
    check_accuracy_test
)

from models import UNET
from monai.networks.nets import UNet
from train import train_epoch
from simulated_lung_dataset_gaussian import get_loaders_simulated
from simulated_lung_dataset_extra import get_loaders

In [4]:
# HYPERPARAMETERS:
LEARNING_RATE = 1e-4
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
PIN_MEMORY = False  # True if DEVICE == 'cuda' else False
LOGGING = True
BATCH_SIZE = 8
NUM_EPOCHS = 50
NUM_WORKERS = 0
IMAGE_SIZE = 256

# train

In [5]:
def model_train(model_input, save_model_path, LOGGING, DEVICE):    
    model = model_input
    loss_fn = BCEWithLogitsLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
    scaler = torch.cuda.amp.GradScaler()

    # training
    train_loader, test_loader, validation_loader = get_loaders()
    for epoch in range(1, NUM_EPOCHS+1):
        loss = train_epoch(train_loader, model, optimizer, loss_fn, scaler, DEVICE)
        if epoch % 5 == 0:
            check_accuracy_val(epoch, validation_loader, model, logging = LOGGING ,device=DEVICE)
        del loss
        torch.cuda.empty_cache() 
        
    checkpoint = {
            "state_dict": model.state_dict(),
            "optimizer": optimizer.state_dict(),
            }
    check_accuracy_test(epoch, test_loader, model, logging = LOGGING ,device=DEVICE)
    save_checkpoint(checkpoint, filename= save_model_path)

# main

In [6]:
def model_train_log(i, model, FEATURES, IMAGE_SIZE, LOGGING, DEVICE = 'cuda'):
    model_save_path = f'/home/alex/Documents/new try/Data/Lung Unet/save_states/Own_Unet_gaussian_test_123.pth.tar' 
    
    #logging
    if LOGGING:
        run = wandb.init(
            project = "Segmentation normal",
            group = "normal",
            job_type = 'normal',
            name = 'monai_UNet',

            # track hyperparameters and run metadata
            config={
                "try no." : i,
                "features" : FEATURES,
                "image_size" : IMAGE_SIZE,
                },
        )
    else:
        pass
 
    
    model_train(
        model,
        model_save_path,
        LOGGING,
        DEVICE)
    
    if LOGGING:
        run.finish()

    torch.cuda.empty_cache() 
   

In [7]:
def run_tests():
    for i in range(3):
        for IMAGE_SIZE in [256]:
            for FEATURES in [(8,16, 32, 64)]:        
                LOGGING = True
                #wn_unet = UNET(in_channels=1,out_channels=1, features= FEATURES).to(DEVICE)
                monai_UNet = UNet(spatial_dims=2, in_channels=1, out_channels=1, channels=FEATURES, strides=(2, 2, 2), bias = False, num_res_units = 0).to(device=DEVICE)
                #load_checkpoint(Own_unet, r'/home/alex/Documents/new try/Data/Lung Unet/save_states/Own_unet_padding_1.pth.tar')
                model_train_log(i, monai_UNet, FEATURES, IMAGE_SIZE, LOGGING, DEVICE)
                

if __name__ == "__main__":
    run_tests()

100%|██████████| 62/62 [00:45<00:00,  1.35it/s, loss=0.498]


Val-Epoch: 5, Acc: 0.87, and Dice score: 0.57, IoU: 0.81, hd: 76.09


100%|██████████| 62/62 [00:44<00:00,  1.39it/s, loss=0.372]


Val-Epoch: 10, Acc: 0.95, and Dice score: 0.85, IoU: 0.83, hd: 25.44


100%|██████████| 62/62 [00:43<00:00,  1.44it/s, loss=0.292]


...

Val-Epoch: 45, Acc: 0.98, and Dice score: 0.93, IoU: 0.93, hd: 14.89


100%|██████████| 62/62 [00:45<00:00,  1.37it/s, loss=0.0833]


Val-Epoch: 50, Acc: 0.98, and Dice score: 0.93, IoU: 0.93, hd: 13.53
Epoch: 50, Acc: 0.97, and Dice score: 0.93, IoU: 0.93, hd: 13.45
=> Saving checkpoint


epoch,▁
test_acc,▁
test_box_iou,▁
test_dice,▁
test_hd,▁
test_loss,▁
epoch,50
test_acc,0.97462
test_box_iou,0.92765
test_dice,0.92907
test_hd,13.44573


: 

In [ ]:
from utils import plot_image_mask_box_pred_box
train_loader, test_loader, validation_loader = get_loaders()
monai_UNet = UNet(spatial_dims=2, in_channels=1, out_channels=1, channels=(4, 8, 16, 32), strides=(2, 2, 2), bias = False, num_res_units = 0).to(device=DEVICE)
load_checkpoint(monai_UNet,r'/home/alex/Documents/new try/Data/Lung Unet/save_states/Monai_Unet_final_1.pth.tar')

for image, target in test_loader:
    plot_image_mask_box_pred_box(monai_UNet, image, target)